In [1]:
!pip install flask-ngrok > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1

from google.colab import drive

drive.mount('/content/drive')

with open('drive/MyDrive/Colab Notebooks/project/static/ngrok_auth.txt') as nf:
    ngrok_auth = nf.read()

!ngrok authtoken $ngrok_auth

from flask import Flask, render_template, request, current_app
import os

from flask_ngrok import run_with_ngrok

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=e0b21c655ae61e1ef4c34f01dec9d88c1098ce3bff1a2db1a9a8c7e2dd23c9f0
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [3]:
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
hyper_param_batch = 1 # 4이상으로 설정 

random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

num_classes = 4
model_name = 'efficientnet-b7'

train_name = 'model1'

# PATH = './scalp_weights/'
PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/' 

# data_train_path ='/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/train' 
# data_validation_path = '/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/validation' 
# data_test_path ='/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/test' 
data_train_path ='/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/train' 
data_validation_path = '/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/validation' 
data_test_path ='/content/drive/MyDrive/Colab Notebooks/train_data/'+train_name+'/test' 

image_size = EfficientNet.get_image_size(model_name)
print(image_size)

model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
model = model.to(device)

def function(x):
    return x.rotate(90)


transforms_train = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                        transforms.RandomHorizontalFlip(p=0.5),
                                        transforms.RandomVerticalFlip(p=0.5),
                                        transforms.Lambda(function),
                                        transforms.RandomRotation(10),
                                        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])

transforms_val = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BICUBIC),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])


train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)

dataloaders, batch_num = {}, {}

dataloaders['train'] = DataLoader(train_data_set,
                                    batch_size=hyper_param_batch,
                                    shuffle=True,
                                    num_workers=2)
dataloaders['val'] = DataLoader(val_data_set,
                                    batch_size=hyper_param_batch,
                                    shuffle=False,
                                    num_workers=2)

batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set)

print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val']))

class_names = train_data_set.classes
print(class_names)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    if __name__ == '__main__':
        start_time = time.time()
        
        since = time.time()
        best_acc = 0.0
        best_model_wts = copy.deepcopy(model.state_dict())
        
        train_loss, train_acc, val_loss, val_acc = [], [], [], []
        
        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
            
            epoch_start = time.time()

            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0
                num_cnt = 0

                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    num_cnt += len(labels)
                    
                if phase == 'train':
                    scheduler.step()
                
                epoch_loss = float(running_loss / num_cnt)
                epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
                
                if phase == 'train':
                    train_loss.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    val_loss.append(epoch_loss)
                    val_acc.append(epoch_acc)
                    
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                    
                if phase == 'val' and epoch_acc > best_acc:
                    best_idx = epoch
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    print('==> best model saved - %d / %.1f'%(best_idx, best_acc))
                    
                epoch_end = time.time() - epoch_start
                
                print('Training epochs {} in {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60, epoch_end % 60))
                print()
                
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

        model.load_state_dict(best_model_wts)
            
        torch.save(model, PATH + 'aram_'+train_name+'.pt')
        torch.save(model.state_dict(), PATH + 'president_aram_'+train_name+'.pt')
        print('model saved')

        end_sec = time.time() - start_time
        end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
        end_time = end_times[0]
        print("end time :", end_time)
        
        return model, best_idx, best_acc, train_loss, train_acc, val_loss, val_acc


criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model.parameters(),lr = 1e-4)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

num_epochs = 2
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)

600


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7
batch_size : 1,  train/val : 2074 / 514
['[원천]미세각질_0.양호', '[원천]미세각질_1.경증', '[원천]미세각질_2.중등도', '[원천]미세각질_3.중증']
Epoch 0/1
----------
train Loss: 1.1918 Acc: 42.5747
Training epochs 0 in 14m 26s

val Loss: 2.4474 Acc: 6.4202
==> best model saved - 0 / 6.4
Training epochs 0 in 15m 37s

Epoch 1/1
----------
train Loss: 1.1895 Acc: 40.3086
Training epochs 1 in 14m 13s

val Loss: 1.7769 Acc: 39.1051
==> best model saved - 1 / 39.1
Training epochs 1 in 15m 7s

Training complete in 30m 44s
Best valid Acc: 1 - 39.1
model saved
end time : 0:30:52


(EfficientNet(
   (_conv_stem): Conv2dStaticSamePadding(
     3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
     (static_padding): ZeroPad2d((0, 1, 0, 1))
   )
   (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_blocks): ModuleList(
     (0): MBConvBlock(
       (_depthwise_conv): Conv2dStaticSamePadding(
         64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
         (static_padding): ZeroPad2d((1, 1, 1, 1))
       )
       (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
       (_se_reduce): Conv2dStaticSamePadding(
         64, 16, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_se_expand): Conv2dStaticSamePadding(
         16, 64, kernel_size=(1, 1), stride=(1, 1)
         (static_padding): Identity()
       )
       (_project_conv): Conv2dStaticSamePadding(
         64, 32, kernel_size=(1, 1), strid